In [1]:
from translation_machine import dataset_mod
import numpy as np
import torch
whole_dataset = dataset_mod.DatasetFromTxt("../data/french_english_dataset/fra.txt")


# Remark : the responsability to split the dataset is done outside of this notebook

idxs_train = np.load("../dataset_splitting/idx_train.npy")
idxs_val = np.load("../dataset_splitting/idx_val.npy")
idxs_test = np.load("../dataset_splitting/idx_test.npy")

train_dataset = torch.utils.data.Subset(whole_dataset,idxs_train)
val_dataset = torch.utils.data.Subset(whole_dataset,idxs_val)
test_dataset = torch.utils.data.Subset(whole_dataset,idxs_test)

/Users/aammor/miniforge3/envs/aa_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from translation_machine import dataset_mod,collate_fn_mod,sentence_mod

from itertools import islice
train_dataset_as_sentence = dataset_mod.SentenceDataSet(train_dataset,
                                                        sentence_mod.EnglishSentence,
                                                        sentence_mod.FrenchSentence)




/Users/aammor/miniforge3/envs/aa_env/lib/python3.8/site-packages/torchtext/data/utils.py:105: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(


In [4]:
language_info = torch.load("../models/language_info.pth")

vocab_src = language_info["english"]["vocab"]
vocab_dst = language_info["french"]["vocab"]
from torchtext.data.utils import get_tokenizer
# we use the same tokenizer for english and 
tokenizer = get_tokenizer('spacy', language='en')
src_tokenizer,dst_tokenizer = tokenizer,tokenizer


In [5]:
from translation_machine import collate_fn_mod
from itertools import cycle

max_length_src,max_length_dst = 100,100
collate_fn = collate_fn_mod.get_collate_fn(max_length_src,max_length_dst)

collate_fn_standard = collate_fn_mod.get_collate_fn_standard(vocab_src,vocab_dst,src_tokenizer,dst_tokenizer,max_length_src,max_length_dst)

In [14]:
batch_size = 30
max_iter = 100

N = min(int(len(train_dataset)/batch_size),max_iter)


In [15]:
from torch.utils.data import DataLoader
train_data_loader = DataLoader(train_dataset_as_sentence,batch_size=batch_size,collate_fn=collate_fn)
train_data_loader_standard = DataLoader(train_dataset,batch_size=batch_size,collate_fn=collate_fn_standard)


In [16]:
%%time

res = [el for el in islice(train_data_loader,100)]

CPU times: user 96.4 ms, sys: 1.3 ms, total: 97.7 ms
Wall time: 96.9 ms


In [17]:
%%time

res = [el for el in islice(train_data_loader_standard,100)]

CPU times: user 90.9 ms, sys: 1.03 ms, total: 92 ms
Wall time: 91.2 ms
